In [1]:
import json
import os
import itertools
import requests
import sqlparse
import re
from IPython.display import HTML, display
import tabulate
import urllib.parse

In [2]:
def get_doc_id(doc_id):
    doc_id = urllib.parse.quote(doc_id)
    url = 'https://next.obudget.org/get/'+doc_id
    print('Getting', url)
    ret = requests.get(url).json()['value']
    assert 'page_title' in ret, repr(ret)
    return ret

In [3]:
def get_table(query, headers, formatters):
    resp = requests.get('https://next.obudget.org/api/query', params={'query': query})
    if resp.status_code != 200:
        raise ValueError(resp.content)
    table = resp.json()
    rows = table['rows']
    ret = []
    if len(rows)>0:
        ret.append(headers)
        ret.extend([[f(x) for f in formatters] for x in rows][:3])
    return ret + [['Total', table['total']]]

In [4]:
def questions():
    base = 'app/descriptors'
    tgt = re.compile('questions.*\.json')
    for root, dirs, fnames in os.walk(base):
        for f in fnames:
            if tgt.match(f):
                yield root.replace(base + '/', ''), os.path.join(root, f)

In [5]:
def parsed_questions():
    for q in questions():
        print(q)
        qq = json.load(open(q[1]))
        for qqq in qq:
            yield q[0], qqq

In [6]:
def parameters(q):
    p = q.get('parameters', {})
    keys = p.keys()
    values = [list(p[k].items()) for k in keys]
    return (zip(keys, prod) for prod in itertools.product(*values))

In [22]:
DOC_IDS = [
    ('budget', [
        'budget/C1/2019',
        'budget/C222/2019',        
        'budget/0020/2019',
        'budget/0000/2019',
        'budget/002043/2019',
        'budget/00204301/2019',
        'budget/0020430101/2019',
    ]),
    ('supports', [
        'supports/0020460604/2016/עירית רחובות/הקצבות'
    ]),
    ('org', [
        'org/company/520039710'
    ]),
    ('contract-spending', [
        'contract-spending/משרד האוצר/4501287236/0083050502',
        'contract-spending/שירות בתי הסוהר/4501041809/0052400102',
    ]),
    ('budget-changes', [
        'national-budget-changes/2015/26-004'
    ]),
    ('tenders/exemptions', [
        'tenders/exemptions/31252/none'
    ])
]

In [23]:
def formatter(mod, h):
    def _f(x, row):
        return str(x) + ' + ' + mod
    return _f

In [24]:
def compose(f, g):
    def _f(x, row):
        return g(f(x, row), row)
    return _f

In [25]:
def getter(h):
    hdr = h
    def _f(x, row):
        return row[hdr]
    return _f

In [26]:
def wrapper(f):
    def _f(row):
        return f('', row)
    return _f

In [31]:
PARAM = re.compile(':([a-z()_]+)$')

def parse_headers(headers):
    _headers = []
    _formatters = []
    for h in headers:
        matches = PARAM.findall(h)
        funcs = []
        while len(matches)>0:
            mod = matches[0]
            h = h[:-(len(mod)+1)]
            funcs.append(formatter(mod, h))
            matches = PARAM.findall(h)
        f = getter(h)
        for g in funcs:
            f = compose(f, g)
        k = wrapper(f)
        _formatters.append(k)
        _headers.append(h)
    return _headers, _formatters
        
    

In [33]:
skip = {'budget'}
params_re = re.compile('(:[a-z_]+)')
for kind, q in parsed_questions():
    pass
print('---------------------')
for kind, q in parsed_questions():
    if kind in skip: continue
    text = q['text']
    query = q['query']
    if isinstance(query, list):
        query = ' '.join(query)
    headers = q.get('headers', [])
    headers, formatters = parse_headers(headers)
    doc_ids = dict(DOC_IDS)[kind]
    for doc_id in doc_ids:
        doc = get_doc_id(doc_id)
        for p in parameters(q):
            params = {}
            updated_text = q['text']
            updated_query = query
            params.update(doc)
            for name, (disp, val) in p:
                params[name] = val
                updated_text = updated_text.replace('<'+name+'>', disp)
            display(HTML('<h3>({}) {}: {}</h3>'.format(kind, doc['page_title'], updated_text)))
            ofs = 0
            while True:
                m = params_re.search(updated_query[ofs:])
                if m is None: break
                key = m.groups(0)[0][1:]
                if key in params:
                    updated_query = updated_query[:(m.start()+ofs)] + str(params[key]) + updated_query[(m.end()+ofs):]
                else:
                    print('skipping parameter', key)
                    ofs = ofs + m.start()+1
            display(HTML('<pre>{}</pre>'.format(sqlparse.format(updated_query, reindent=True, keyword_case='upper'))))
            display(HTML(tabulate.tabulate(get_table(updated_query, headers, formatters), tablefmt='html')))


('budget', 'app/descriptors/budget/questions.json')
('budget', 'app/descriptors/budget/questions.spending.json')
('budget-changes', 'app/descriptors/budget-changes/questions.json')
('contract-spending', 'app/descriptors/contract-spending/questions.json')
('org', 'app/descriptors/org/questions.json')
('people', 'app/descriptors/people/questions.json')
('reports/ngo-activity-report', 'app/descriptors/reports/ngo-activity-report/questions.json')
('reports/ngo-district-report', 'app/descriptors/reports/ngo-district-report/questions.json')
('supports', 'app/descriptors/supports/questions.json')
('tenders/central', 'app/descriptors/tenders/central/questions.json')
('tenders/exemptions', 'app/descriptors/tenders/exemptions/questions.json')
('tenders/office', 'app/descriptors/tenders/office/questions.json')
('tenders.old/central', 'app/descriptors/tenders.old/central/questions.json')
('tenders.old/exemptions', 'app/descriptors/tenders.old/exemptions/questions.json')
('tenders.old/office', 'app

שם הספק,כותרת התקשרות,הגורם המזמין,סוג הרכש,סטטוס התקשרות,סכום שאושר,סכום ששולם,מטבע,תאריך הזמנה,תאריך עדכון
"אינטרנט בינת בע""מ",התקנה ותחזוקה של כלובים אתר כלים שלובים,רשות האכיפה והגבייה,תקנה 14ב - מכרז מרכזי,True,720883.78 + number,635242.2 + number,ILS,2015-04-21,2018-07-11
"אינטרנט בינת בע""מ",תשלום חשמל,רשות האוכלוסין וההגירה,תקנה 14ב - מכרז מרכזי,True,339300.0 + number,0.0 + number,ILS,2015-11-11,2018-07-23
"אינטרנט בינת בע""מ",אירוח DRP,החטיבה להתיישבות בהסתדרות הציונית העולמית,תקנה 14ב - מכרז מרכזי,True,195290.88 + number,175933.42 + number,ILS,2015-11-30,2018-02-19
Total,8,,,,,,,,


שם הספק,כותרת התקשרות,הגורם המזמין,סוג הרכש,סטטוס התקשרות,סכום שאושר,סכום ששולם,מטבע,תאריך הזמנה,תאריך עדכון
"אינטרנט בינת בע""מ","לשימוש המשרד לבט""פ",המשרד לביטחון הפנים,תקנה 14ב - מכרז מרכזי,True,507247.69 + number,333174.4 + number,ILS,2016-03-16,2018-06-07
"אינטרנט בינת בע""מ",שירות תחזוקה,רשות המים והביוב,תקנה 14ב - מכרז מרכזי,True,208241.28 + number,55184.14 + number,ILS,2016-11-21,2018-01-23
"אינטרנט בינת בע""מ",שוטף,משרד המדע והטכנולוגיה,תקנה 14ב - מכרז מרכזי,True,73168.92 + number,68187.04 + number,ILS,2016-11-28,2018-07-16
Total,9,,,,,,,,


שם הספק,כותרת התקשרות,הגורם המזמין,סוג הרכש,סטטוס התקשרות,סכום שאושר,סכום ששולם,מטבע,תאריך הזמנה,תאריך עדכון
"אינטרנט בינת בע""מ",ארונות שרתים לחדר שרתים,שירות התעסוקה הישראלי,תקנה 14ב - מכרז מרכזי,True,362459.45 + number,260566.19 + number,ILS,2017-01-30,2018-01-03
"אינטרנט בינת בע""מ",תשלום חצי שנתי עבור ארונות,הרשות הארצית לכבאות והצלה,תקנה 14ב - מכרז מרכזי,True,55373.76 + number,0.0 + number,ILS,2017-12-13,2018-07-10
"אינטרנט בינת בע""מ",אירוח לתשתיות DATA CENTER המרכזי של המשרד לשוויון,המשרד לשוויון חברתי,תקנה 14ב - מכרז מרכזי,True,134747.5 + number,9228.96 + number,ILS,2017-05-15,2017-09-19
Total,26,,,,,,,,


שם הספק,כותרת התקשרות,הגורם המזמין,סוג הרכש,סטטוס התקשרות,סכום שאושר,סכום ששולם,מטבע,תאריך הזמנה,תאריך עדכון
"אינטרנט בינת בע""מ",שרותיאבטחחה למערכות מיחשוב,משרד התרבות והספורט,תקנה 14ב - מכרז מרכזי,True,92720.16 + number,44727.17 + number,ILS,2018-01-28,2018-07-23
"אינטרנט בינת בע""מ","רכישת שטחי DC לטובת אירוח מערך המחשוב שישמש להתקנת אתר ה DR שישרת את רשל""",משרד המשפטים,תקנה 14ב - מכרז מרכזי,True,303835.66 + number,103824.04 + number,ILS,2018-02-22,2018-07-23
"אינטרנט בינת בע""מ",שרותי פרסום,משרד התרבות והספורט,פטור ממכרז,True,292.5 + number,292.5 + number,ILS,2018-04-15,2018-07-23
Total,10,,,,,,,,


...

Total,0


Total,0


Total,0


Getting https://next.obudget.org/get/org/company/520039710


ValueError: b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request.  Either the server is overloaded or there is an error in the application.</p>\n'